# Module 12: Complete FastAPI Learning Path

This notebook demonstrates all applications organized by learning level with practical examples and explanations.

## Structure
1. **Beginner Level** - Todo App (User, Todo)
2. **Intermediate Level** - Blog API (User, Post, Comment)
3. **Advanced Level** - ML Registry (Complex production application)

---

# Part 1: Beginner Level - Todo App

## Learning Objectives
- Basic FastAPI routes
- User authentication with JWT
- Simple database models (User, Todo)
- CRUD operations
- Basic testing

## Application Overview
A simple Todo application where:
- Users register and login
- Users can create, read, update, delete their todos
- Todos have: id, title, description, completed status

## Database Schema
```
User
├── id (Integer, PK)
├── email (String, unique)
├── username (String, unique)
├── hashed_password (String)
├── is_active (Boolean)
└── todos (Relationship)

Todo
├── id (Integer, PK)
├── title (String)
├── description (String)
├── completed (Boolean)
├── user_id (Integer, FK)
└── user (Relationship)
```

## Theory: FastAPI Basics

### What is FastAPI?
- Modern, fast web framework for building APIs
- Built on Starlette (web) and Pydantic (validation)
- Async by default (non-blocking I/O)
- Automatic interactive API documentation
- Type hints for validation and documentation

### Key Concepts

#### 1. Routes (Path Operations)
```python
@app.get("/todos")  # HTTP GET at /todos
async def get_todos(user_id: int):
    return [...]  # Returns JSON
```

#### 2. Path Parameters
```python
@app.get("/todos/{todo_id}")  # {todo_id} is variable
async def get_todo(todo_id: int):
    return {"id": todo_id}
```

#### 3. Query Parameters
```python
@app.get("/todos")  # ?skip=0&limit=10
async def list_todos(skip: int = 0, limit: int = 10):
    return todos[skip:skip+limit]
```

#### 4. Request Body with Pydantic
```python
class TodoCreate(BaseModel):
    title: str
    description: str = None

@app.post("/todos")
async def create_todo(todo: TodoCreate):
    # FastAPI validates and converts JSON to TodoCreate object
    return {"title": todo.title, ...}
```

#### 5. Dependency Injection
```python
async def get_current_user(
    token: str = Header(...),
    db: AsyncSession = Depends(get_db)
):
    # FastAPI injects dependencies automatically
    return user

@app.get("/todos")
async def get_todos(current_user: User = Depends(get_current_user)):
    return current_user.todos
```

#### 6. Response Model
```python
class TodoResponse(BaseModel):
    id: int
    title: str
    completed: bool

@app.get("/todos", response_model=List[TodoResponse])
async def get_todos():
    # FastAPI validates response matches TodoResponse
    return todos
```

## API Examples for Todo App

### Prerequisites
Before running these examples, start the Todo app:

```bash
cd beginner_edition/todo_app
docker-compose up
# Wait for "Application startup complete"
```

Then run this notebook.

In [ ]:
import requests
import json
from typing import Optional

# API Configuration
API_URL = "http://localhost:8000"
headers = {"Content-Type": "application/json"}

print("✅ Imports successful. Ready to test Todo App API.")
print(f"📍 Target: {API_URL}")

### 1. User Registration

**Concept:** Create a new user account

**API Endpoint:** `POST /auth/register`

**Request Body:**
```json
{
  "email": "user@example.com",
  "username": "user",
  "password": "pass123"
}
```

**Why?** Users need accounts before they can create todos.

In [ ]:
# Register a new user
user_data = {
    "email": "alice@example.com",
    "username": "alice",
    "password": "SecurePass123!"
}

response = requests.post(
    f"{API_URL}/auth/register",
    json=user_data,
    headers=headers
)

print(f"Status: {response.status_code}")
print(f"Response: {json.dumps(response.json(), indent=2)}")

# Store for later use
user_id = response.json()["id"]
username = user_data["username"]
password = user_data["password"]

### 2. User Login

**Concept:** Get JWT token for authentication

**API Endpoint:** `POST /auth/login`

**Request Body:**
```
username=user&password=pass123
```
Note: Form data, not JSON

**Response:**
```json
{
  "access_token": "eyJ0eXAi...",
  "token_type": "bearer"
}
```

**Why?** JWT token is used to authenticate subsequent requests.

In [ ]:
# Login to get JWT token
login_data = {
    "username": username,
    "password": password
}

response = requests.post(
    f"{API_URL}/auth/login",
    data=login_data  # Form data, not JSON
)

print(f"Status: {response.status_code}")
login_response = response.json()
print(f"Response: {json.dumps(login_response, indent=2)}")

# Store token for authenticated requests
access_token = login_response["access_token"]
auth_headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

print(f"\n✅ Token obtained: {access_token[:50]}...")

### 3. Create a Todo

**Concept:** Create a new todo item for the current user

**API Endpoint:** `POST /todos`

**Authentication:** Requires JWT token in `Authorization` header

**Request Body:**
```json
{
  "title": "Learn FastAPI",
  "description": "Complete the FastAPI module",
  "completed": false
}
```

**Why?** Users create todos as their main use case.

In [ ]:
# Create multiple todos
todos_to_create = [
    {
        "title": "Learn FastAPI",
        "description": "Complete the FastAPI module and understand routes, models, auth",
        "completed": False
    },
    {
        "title": "Build a project",
        "description": "Create a todo API using FastAPI",
        "completed": False
    },
    {
        "title": "Deploy to cloud",
        "description": "Deploy the todo app to AWS or DigitalOcean",
        "completed": False
    }
]

created_todos = []

for todo_data in todos_to_create:
    response = requests.post(
        f"{API_URL}/todos",
        json=todo_data,
        headers=auth_headers
    )
    
    print(f"Status: {response.status_code}")
    created_todo = response.json()
    created_todos.append(created_todo)
    print(f"✅ Created: {created_todo['title']} (ID: {created_todo['id']})\n")

### 4. Get All Todos

**Concept:** List all todos for the current user

**API Endpoint:** `GET /todos`

**Query Parameters:**
- `skip` (int): Number of items to skip (pagination)
- `limit` (int): Maximum items to return (pagination)
- `completed` (bool): Filter by completion status

**Why?** Users want to see all their todos.

In [ ]:
# Get all todos
response = requests.get(
    f"{API_URL}/todos",
    headers=auth_headers
)

print(f"Status: {response.status_code}")
todos = response.json()
print(f"\nTotal todos: {len(todos)}")
print(f"\nAll todos:")
for todo in todos:
    status = "✅" if todo["completed"] else "❌"
    print(f"{status} [{todo['id']}] {todo['title']}")
    if todo["description"]:
        print(f"    → {todo['description']}")

### 5. Get Single Todo

**Concept:** Retrieve details of a specific todo

**API Endpoint:** `GET /todos/{todo_id}`

**Path Parameter:**
- `todo_id` (int): The ID of the todo to retrieve

**Why?** Users want to view details of a specific todo.

In [ ]:
# Get a specific todo
if created_todos:
    todo_id = created_todos[0]["id"]
    
    response = requests.get(
        f"{API_URL}/todos/{todo_id}",
        headers=auth_headers
    )
    
    print(f"Status: {response.status_code}")
    todo = response.json()
    print(f"\nTodo Details:")
    print(json.dumps(todo, indent=2))
else:
    print("No todos created yet. Please create a todo first.")

### 6. Update a Todo

**Concept:** Modify an existing todo

**API Endpoint:** `PUT /todos/{todo_id}`

**Request Body:**
```json
{
  "title": "Updated title",
  "description": "Updated description",
  "completed": true
}
```

**Why?** Users need to update todo details and mark them complete.

In [ ]:
# Update a todo - mark first one as completed
if created_todos:
    todo_id = created_todos[0]["id"]
    
    update_data = {
        "title": created_todos[0]["title"],
        "description": created_todos[0]["description"],
        "completed": True  # Mark as completed
    }
    
    response = requests.put(
        f"{API_URL}/todos/{todo_id}",
        json=update_data,
        headers=auth_headers
    )
    
    print(f"Status: {response.status_code}")
    updated_todo = response.json()
    print(f"\n✅ Updated todo:")
    print(f"Title: {updated_todo['title']}")
    print(f"Completed: {updated_todo['completed']}")

### 7. Delete a Todo

**Concept:** Remove a todo from the database

**API Endpoint:** `DELETE /todos/{todo_id}`

**Path Parameter:**
- `todo_id` (int): The ID of the todo to delete

**Response:**
```json
{
  "id": 1,
  "message": "Todo deleted successfully"
}
```

**Why?** Users want to remove completed or unwanted todos.

In [ ]:
# Delete a todo - remove the second one
if len(created_todos) > 1:
    todo_id = created_todos[1]["id"]
    
    response = requests.delete(
        f"{API_URL}/todos/{todo_id}",
        headers=auth_headers
    )
    
    print(f"Status: {response.status_code}")
    print(f"Response: {json.dumps(response.json(), indent=2)}")
    
    # Get all todos again to show it was deleted
    response = requests.get(
        f"{API_URL}/todos",
        headers=auth_headers
    )
    print(f"\nTodos after deletion: {len(response.json())}")
else:
    print("Not enough todos to delete. Please create at least 2 todos.")

---

# Part 2: Intermediate Level - Blog API

## Learning Objectives
- Database relationships (One-to-Many)
- Pagination and filtering
- Complex queries
- Multi-user authorization

## Application Overview
A blog application where:
- Users create blog posts
- Users can comment on posts
- Posts can be filtered by author or status
- Pagination support for large lists

## Database Schema
```
User
├── id (Integer, PK)
├── email (String, unique)
├── username (String, unique)
├── hashed_password (String)
├── posts (Relationship: List[Post])
└── comments (Relationship: List[Comment])

Post
├── id (Integer, PK)
├── title (String)
├── content (String)
├── published (Boolean)
├── author_id (Integer, FK)
├── author (Relationship: User)
└── comments (Relationship: List[Comment])

Comment
├── id (Integer, PK)
├── content (String)
├── author_id (Integer, FK)
├── post_id (Integer, FK)
├── author (Relationship: User)
└── post (Relationship: Post)
```

## Theory: Database Relationships

### What are Relationships?
Relationships define how tables connect to each other.

### One-to-Many Relationship
```
User (One) ────→ (Many) Posts
One user can have many posts
Each post belongs to exactly one user
```

**In SQLAlchemy 2.0:**
```python
class User(Base):
    __tablename__ = "users"
    id: Mapped[int] = mapped_column(primary_key=True)
    posts: Mapped[List["Post"]] = relationship(back_populates="author")

class Post(Base):
    __tablename__ = "posts"
    id: Mapped[int] = mapped_column(primary_key=True)
    author_id: Mapped[int] = mapped_column(ForeignKey("users.id"))
    author: Mapped[User] = relationship(back_populates="posts")
```

### Querying with Relationships
```python
# Get user with all posts
user = await db.get(User, user_id)
user.posts  # Access posts without additional query (with eager loading)

# Get post with author
post = await db.get(Post, post_id)
post.author.username  # Access author
```

### Pagination
Pagination divides large result sets into pages:

```python
# Get 10 items per page
@app.get("/posts")
async def list_posts(skip: int = 0, limit: int = 10):
    # skip=0, limit=10 → items 0-9
    # skip=10, limit=10 → items 10-19
    posts = await db.execute(
        select(Post).offset(skip).limit(limit)
    )
    return posts.scalars().all()
```

### Filtering
Filtering narrows results based on criteria:

```python
# Filter by author
@app.get("/posts")
async def list_posts(author_id: int = None, published: bool = None):
    query = select(Post)
    if author_id:
        query = query.where(Post.author_id == author_id)
    if published is not None:
        query = query.where(Post.published == published)
    posts = await db.execute(query)
    return posts.scalars().all()
```

## API Examples for Blog API

### Prerequisites
Before running these examples, start the Blog API:

```bash
cd intermediate_edition/blog_api
docker-compose up
# Wait for "Application startup complete"
```

Then update the API URL and run the examples below.

In [ ]:
# Update API URL for Blog API
API_URL = "http://localhost:8001"  # Blog API runs on port 8001
print(f"📍 Switched to Blog API: {API_URL}")

# Register a blog user
blog_user_data = {
    "email": "blogger@example.com",
    "username": "blogger",
    "password": "BlogPass123!"
}

response = requests.post(
    f"{API_URL}/auth/register",
    json=blog_user_data,
    headers=headers
)

print(f"\nRegistration Status: {response.status_code}")
blogger_id = response.json()["id"]
print(f"✅ Blogger registered (ID: {blogger_id})")

# Login
response = requests.post(
    f"{API_URL}/auth/login",
    data={"username": blog_user_data["username"], "password": blog_user_data["password"]}
)

blog_token = response.json()["access_token"]
blog_auth_headers = {
    "Authorization": f"Bearer {blog_token}",
    "Content-Type": "application/json"
}

print(f"✅ Logged in successfully")

### 1. Create Blog Posts

**Concept:** Create posts that users can comment on

**API Endpoint:** `POST /posts`

**Request Body:**
```json
{
  "title": "Post Title",
  "content": "Post content here",
  "published": true
}
```

In [ ]:
# Create some blog posts
posts_data = [
    {
        "title": "Getting Started with FastAPI",
        "content": "FastAPI is a modern web framework for building APIs with Python. It's fast, easy to learn, and perfect for beginners.",
        "published": True
    },
    {
        "title": "Database Design Best Practices",
        "content": "When designing databases, consider normalization, indexing, and relationships between tables.",
        "published": True
    },
    {
        "title": "Draft: Advanced FastAPI Patterns",
        "content": "This is still a draft post about advanced patterns.",
        "published": False  # Unpublished post
    }
]

created_posts = []

for post_data in posts_data:
    response = requests.post(
        f"{API_URL}/posts",
        json=post_data,
        headers=blog_auth_headers
    )
    
    created_post = response.json()
    created_posts.append(created_post)
    status = "📝" if created_post["published"] else "🔒"
    print(f"{status} Created post: {created_post['title']} (ID: {created_post['id']})")

### 2. Get Posts with Pagination

**Concept:** Retrieve posts with pagination support

**API Endpoint:** `GET /posts`

**Query Parameters:**
- `skip` (int): Number of posts to skip
- `limit` (int): Maximum posts to return

In [ ]:
# Get all published posts with pagination
response = requests.get(
    f"{API_URL}/posts?skip=0&limit=10",
    headers=blog_auth_headers
)

posts = response.json()
print(f"📚 Retrieved {len(posts)} posts:")
for post in posts:
    status = "📝" if post.get("published") else "🔒"
    print(f"\n{status} {post['title']} (ID: {post['id']})")
    print(f"   Author: {post['author']['username']}")
    print(f"   Preview: {post['content'][:80]}...")

### 3. Filter Posts by Author

**Concept:** Show posts filtered by a specific author

**API Endpoint:** `GET /posts?author_id={author_id}`

In [ ]:
# Filter posts by author
response = requests.get(
    f"{API_URL}/posts?author_id={blogger_id}",
    headers=blog_auth_headers
)

author_posts = response.json()
print(f"📚 Posts by author (ID: {blogger_id}): {len(author_posts)}")
for post in author_posts:
    print(f"  • {post['title']}")

### 4. Add Comments to Posts

**Concept:** Add comments showing One-to-Many relationship (Post → Comments)

**API Endpoint:** `POST /posts/{post_id}/comments`

In [ ]:
# Add comments to the first post
if created_posts:
    post_id = created_posts[0]["id"]
    
    comments = [
        "Great introduction to FastAPI!",
        "This helped me understand the basics. Thanks!",
        "Looking forward to more advanced posts."
    ]
    
    for comment_text in comments:
        response = requests.post(
            f"{API_URL}/posts/{post_id}/comments",
            json={"content": comment_text},
            headers=blog_auth_headers
        )
        
        comment = response.json()
        print(f"💬 Comment added: {comment['content']} (by {comment['author']['username']})")

### 5. Get Post with Comments

**Concept:** Retrieve a post with all its comments (eager loading)

**API Endpoint:** `GET /posts/{post_id}`

In [ ]:
# Get post with comments
if created_posts:
    post_id = created_posts[0]["id"]
    
    response = requests.get(
        f"{API_URL}/posts/{post_id}",
        headers=blog_auth_headers
    )
    
    post = response.json()
    print(f"📝 Post: {post['title']}")
    print(f"   Author: {post['author']['username']}")
    print(f"   Content: {post['content']}")
    print(f"\n💬 Comments ({len(post.get('comments', []))})")
    for comment in post.get("comments", []):
        print(f"   • {comment['author']['username']}: {comment['content']}")

---

# Part 3: Advanced Level - ML Registry App

## Learning Objectives
- Complex database schemas (5+ models)
- Advanced relationships and queries
- File storage integration
- Database migrations
- Production patterns and best practices

## Application Overview
A complete ML Model Registry application for managing machine learning models:
- Register and track ML models
- Version control for models
- Track experiments
- Store model files
- User access control

## Database Schema (Simplified)
```
User
├── id, email, username, password
├── models (created by user)
└── experiments (run by user)

MLModel
├── id, name, description, framework (TensorFlow, PyTorch, etc.)
├── owner_id (FK to User)
├── versions (List[ModelVersion])
└── experiments (List[Experiment])

ModelVersion
├── id, version_number, description
├── model_id (FK to MLModel)
├── file_path (where model is stored)
└── metrics (accuracy, loss, etc.)

Experiment
├── id, name, description, status
├── owner_id (FK to User)
├── model_id (FK to MLModel)
└── metrics (results)
```

## Theory: Advanced Patterns

### Complex Relationships
Advanced applications have many interrelated models:

```python
class User(Base):
    models: Mapped[List["MLModel"]] = relationship()
    experiments: Mapped[List["Experiment"]] = relationship()

class MLModel(Base):
    owner_id: Mapped[int] = mapped_column(ForeignKey("users.id"))
    owner: Mapped[User] = relationship(back_populates="models")
    versions: Mapped[List["ModelVersion"]] = relationship()
    experiments: Mapped[List["Experiment"]] = relationship()

class ModelVersion(Base):
    model_id: Mapped[int] = mapped_column(ForeignKey("mlmodels.id"))
    model: Mapped[MLModel] = relationship(back_populates="versions")
```

### File Storage
Store model files separately (MinIO, S3):

```python
class ModelVersion(Base):
    file_key: str  # Key in MinIO/S3
    file_size: int
    file_path: str  # URL to download
```

### Database Migrations
Track schema changes over time with Alembic:

```bash
alembic init migrations/
alembic revision --autogenerate -m "Add MLModel table"
alembic upgrade head  # Apply migrations
```

### API Versioning
Support multiple API versions:

```python
app.include_router(v1_routes.router, prefix="/api/v1")
app.include_router(v2_routes.router, prefix="/api/v2")
```

**Note:** Todo App and ML Registry both use port 8000 by default.
Stop the Todo App before starting ML Registry, or change ports in docker-compose.


## API Examples for ML Registry


In [ ]:
# Update API URL for ML Registry
API_URL = "http://localhost:8000/api/v1"
headers = {"Content-Type": "application/json"}

print(f"📍 Target: {API_URL}")


### 1. User Registration


In [ ]:
# Register a new user
user_data = {
    "email": "ml_user@example.com",
    "username": "ml_user",
    "password": "SecurePass123!"
}

response = requests.post(
    f"{API_URL}/auth/register",
    json=user_data,
    headers=headers
)

print(f"Status: {response.status_code}")
print(f"Response: {json.dumps(response.json(), indent=2)}")

username = user_data["username"]
password = user_data["password"]


### 2. User Login


In [ ]:
# Login to get access + refresh tokens
login_data = {
    "username": username,
    "password": password
}

response = requests.post(
    f"{API_URL}/auth/login",
    data=login_data
)

print(f"Status: {response.status_code}")
login_response = response.json()
print(f"Response: {json.dumps(login_response, indent=2)}")

access_token = login_response["access_token"]
auth_headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

print(f"\n✅ Token obtained: {access_token[:50]}...")


### 3. Create ML Model


In [ ]:
# Create a model record
model_data = {
    "name": "Fraud Detection Model",
    "framework": "sklearn",
    "task_type": "classification",
    "accuracy": 0.97
}

response = requests.post(
    f"{API_URL}/models/",
    json=model_data,
    headers=auth_headers
)

print(f"Status: {response.status_code}")
model_response = response.json()
print(f"Response: {json.dumps(model_response, indent=2)}")

model_id = model_response.get("id")


### 4. List Models


In [ ]:
# List all models
response = requests.get(
    f"{API_URL}/models/",
    headers=auth_headers
)

print(f"Status: {response.status_code}")
print(f"Response: {json.dumps(response.json(), indent=2)}")


### 5. Upload Model File


In [ ]:
# Create a small dummy file and upload it
file_path = "model.pkl"
with open(file_path, "wb") as f:
    f.write(b"model-bytes")

with open(file_path, "rb") as f:
    response = requests.post(
        f"{API_URL}/files/upload",
        headers={"Authorization": auth_headers["Authorization"]},
        files={"file": f}
    )

print(f"Status: {response.status_code}")
print(f"Response: {json.dumps(response.json(), indent=2)}")


## Comparison: Beginner → Intermediate → Advanced

| Aspect | Beginner (Todo) | Intermediate (Blog) | Advanced (ML Registry) |
|--------|-----------------|---------------------|------------------------|
| Models | 2 | 3 | 5+ |
| Relationships | Simple | One-to-Many | Complex chains |
| Database Size | Small | Medium | Large |
| Queries | Simple | Medium | Complex with joins |
| Files | None | None | Yes (MinIO) |
| Migrations | No | No | Yes (Alembic) |
| API Versions | 1 | 1 | 2+ (v1, v2) |
| Tests | Basic | Medium | Comprehensive |
| Performance Concerns | No | Some | Yes |
| Study Time | 2-3h | 3-4h | 5-6h |

---

## Summary: Learning Path

### What You've Learned

**Beginner Level:**
- ✅ Basic FastAPI routing
- ✅ User authentication with JWT
- ✅ Simple database models
- ✅ CRUD operations

**Intermediate Level:**
- ✅ Database relationships (One-to-Many)
- ✅ Pagination and filtering
- ✅ Complex queries
- ✅ Multi-user systems

**Advanced Level:**
- ✅ Complex schemas with 5+ models
- ✅ File storage integration
- ✅ Database migrations
- ✅ Production patterns
- ✅ API versioning

### Next Steps

1. **Run the applications locally**
   - Follow the setup instructions in each edition's README
   - Use the /docs endpoint to explore APIs

2. **Study the code**
   - Read main.py in each app
   - Understand models.py and relationships
   - Review auth.py for security patterns

3. **Extend the applications**
   - Add new features
   - Create new endpoints
   - Modify database schema

4. **Build your own**
   - Apply what you've learned
   - Create a FastAPI application
   - Deploy it

### Resources

- [FastAPI Documentation](https://fastapi.tiangolo.com/)
- [SQLAlchemy 2.0 Docs](https://docs.sqlalchemy.org/)
- [Pydantic Guide](https://docs.pydantic.dev/)
- [JWT.io](https://jwt.io/)

---

**Congratulations on completing Module 12! You now have practical FastAPI skills from beginner to advanced level.** 🎓